In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt

# Load dataset
def load_dataset(file_path, label):
    data = pd.read_csv(file_path)
    # Thêm nhãn vào dữ liệu
    data['label'] = label
    return data

# Đọc dữ liệu từ 3 file và thêm nhãn tương ứng (bỏ staying_data)
falling_data = load_dataset('dataset/falling_dataset.csv', 'falling')
walking_data = load_dataset('dataset/walking_dataset.csv', 'walking')
normal_data = load_dataset('dataset/normal_dataset.csv', 'normal')

# Gộp dữ liệu lại thành 1 DataFrame
all_data = pd.concat([falling_data, walking_data, normal_data], ignore_index=True)
all_data.columns = all_data.columns.str.strip()

# Vẽ biểu đồ của dữ liệu từng file
plt.figure(figsize=(12, 6))

# Vẽ gia tốc X, Y, Z của dữ liệu falling
plt.subplot(3, 1, 1)
plt.plot(falling_data[['accelX', 'accelY', 'accelZ']])
plt.title('Falling Data Acceleration')

# Vẽ gia tốc X, Y, Z của dữ liệu walking
plt.subplot(3, 1, 2)
plt.plot(walking_data[['accelX', 'accelY', 'accelZ']])
plt.title('Walking Data Acceleration')

# Vẽ gia tốc X, Y, Z của dữ liệu normal
plt.subplot(3, 1, 3)
plt.plot(normal_data[['accelX', 'accelY', 'accelZ']])
plt.title('Normal Data Acceleration')

plt.tight_layout()
plt.show()
"""))cells.append(nbf.new_code_cell("""
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load dataset
def load_dataset(file_path, label):
    data = pd.read_csv(file_path)
    # Thêm nhãn vào dữ liệu
    data['label'] = label
    return data

# Đọc dữ liệu từ 3 file và thêm nhãn tương ứng (bỏ staying_data)
falling_data = load_dataset('dataset/falling_dataset.csv', 'falling')
walking_data = load_dataset('dataset/walking_dataset.csv', 'walking')
normal_data = load_dataset('dataset/normal_dataset.csv', 'normal')

# Gộp dữ liệu lại thành 1 DataFrame
all_data = pd.concat([falling_data, walking_data, normal_data], ignore_index=True)
all_data.columns = all_data.columns.str.strip()

# Vẽ biểu đồ của dữ liệu từng file
plt.figure(figsize=(12, 6))

# Vẽ gia tốc X, Y, Z của dữ liệu falling
plt.subplot(3, 1, 1)
plt.plot(falling_data[['accelX', 'accelY', 'accelZ']])
plt.title('Falling Data Acceleration')

# Vẽ gia tốc X, Y, Z của dữ liệu walking
plt.subplot(3, 1, 2)
plt.plot(walking_data[['accelX', 'accelY', 'accelZ']])
plt.title('Walking Data Acceleration')

# Vẽ gia tốc X, Y, Z của dữ liệu normal
plt.subplot(3, 1, 3)
plt.plot(normal_data[['accelX', 'accelY', 'accelZ']])
plt.title('Normal Data Acceleration')

plt.tight_layout()
plt.show()

ImportError: Traceback (most recent call last):
  File "C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
# Chia dữ liệu thành X (features) và y (labels)
X = all_data[['accelX', 'accelY', 'accelZ', 'gyroX', 'gyroY', 'gyroZ']].values
y = all_data['label'].values

# Chuyển nhãn thành số (encoding)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuyển đổi dữ liệu thành dạng 3D (số mẫu, số đặc trưng, 1)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Mô hình CNN 1D
model = models.Sequential([
    # Tăng số lượng filters trong lớp Conv1D đầu tiên
    layers.Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1), padding='same'),
    layers.MaxPooling1D(2),

    # Tăng số lượng filters trong lớp Conv1D thứ hai
    layers.Conv1D(128, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(2),

    # Thêm lớp Dense với nhiều neuron hơn
    layers.Flatten(),
    layers.Dense(64, activation='relu'),  # Tăng số lượng neuron ở lớp Dense
    layers.Dense(3, activation='softmax')  # 3 nhãn: falling, walking, normal
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Lưu mô hình
model.save('fall_detection_model.h5')
print("Saved model")

AttributeError: module 'tensorflow' has no attribute 'keras'

In [ ]:
# Hiển thị độ chính xác kiểm thử
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc:.4f}')

# Vẽ đồ thị độ chính xác trong quá trình huấn luyện
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

# Vẽ đồ thị mất mát trong quá trình huấn luyện
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()